In [1]:
import os, sys

PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '.'))
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR)

/workspaces/street_tree


In [2]:
import joblib
import torch
import pandas as pd
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from src.modeling import TabularNN
from src.config import PATH_MODELS, PATH_PROCESSED

2025-03-23 11:23:26,064 - src.utils - INFO - Logger is ready
2025-03-23 11:23:26,065 - src.utils - INFO - This is a test log message.
2025-03-23 11:23:26,680 - src.utils - INFO - PROJECT_DIR: /workspaces/street_tree
2025-03-23 11:23:26,681 - src.utils - INFO - PATH_LOGS: /workspaces/street_tree/logs
2025-03-23 11:23:26,682 - src.utils - INFO - PATH_DATA: /workspaces/street_tree/data
2025-03-23 11:23:26,683 - src.utils - INFO - PATH_RAW: /workspaces/street_tree/data/raw
2025-03-23 11:23:26,683 - src.utils - INFO - PATH_PROCESSED: /workspaces/street_tree/data/processed
2025-03-23 11:23:26,684 - src.utils - INFO - PATH_MODELS: /workspaces/street_tree/models
2025-03-23 11:23:26,685 - src.utils - INFO - PATH_REPORTS: /workspaces/street_tree/reports
2025-03-23 11:23:26,685 - src.utils - INFO - PATH_DOCS: /workspaces/street_tree/docs
2025-03-23 11:23:26,686 - src.utils - INFO - PATH_SRC: /workspaces/street_tree/src
2025-03-23 11:23:26,687 - src.utils - INFO - output_path: /workspaces/street_t

In [3]:
app = FastAPI()

In [4]:
# Пути к моделям и энкодерам
MODEL_PATH = f"{PATH_MODELS}/tabular_model.pth"
SCALER_PATH = f"{PATH_MODELS}/scaler.pkl"
ENCODERS_PATH = f"{PATH_MODELS}/label_encoders.pkl"


In [5]:
TARGET_ENCODER_PATH = f"{PATH_MODELS}/label_encoders_target.pkl"
target_mapping = joblib.load(TARGET_ENCODER_PATH)
inverse_target_mapping = {v: k for k, v in target_mapping.items()}
inverse_target_mapping

{0: 'Poor', 1: 'Fair', 2: 'Good'}

In [1]:
# Пути к моделям и энкодерам
MODEL_PATH = f"{PATH_MODELS}/tabular_model.pth"
SCALER_PATH = f"{PATH_MODELS}/scaler.pkl"
ENCODERS_PATH = f"{PATH_MODELS}/label_encoders.pkl"

# Dummy data for loading the model
dummy_X = pd.DataFrame(np.zeros((1, 27)))  # Replace 27 with your actual input dimension
dummy_y = pd.Series([0])

# Загрузка моделей и энкодеров
model = TabularNN.load_model(MODEL_PATH, dummy_X, dummy_y, dummy_X, dummy_y)

TARGET_ENCODER_PATH = f"{PATH_MODELS}/label_encoders_target.pkl"
target_mapping = joblib.load(TARGET_ENCODER_PATH)
inverse_target_mapping = {v: k for k, v in target_mapping.items()}
inverse_target_mapping

NameError: name 'PATH_MODELS' is not defined

In [ ]:
# Обратное отображение для целевого признака
scaler = joblib.load(SCALER_PATH)
label_encoders = joblib.load(ENCODERS_PATH)
target_mapping = joblib.load(TARGET_ENCODER_PATH)
inverse_target_mapping = {v: k for k, v in target_mapping.items()}
inverse_target_mapping

{0: 'Poor', 1: 'Fair', 2: 'Good'}

In [ ]:
df = pd.read_csv(f"{PATH_PROCESSED}/test.csv")
df.columns

Index(['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam',
       'curb_loc', 'status', 'spc_latin', 'spc_common', 'steward', 'guards',
       'sidewalk', 'user_type', 'problems', 'root_stone', 'root_grate',
       'root_other', 'trunk_wire', 'trnk_light', 'trnk_other', 'brch_light',
       'brch_shoe', 'brch_other', 'address', 'postcode', 'zip_city',
       'community board', 'borocode', 'borough', 'cncldist', 'st_assem',
       'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude',
       'longitude', 'x_sp', 'y_sp', 'council district', 'census tract', 'bin',
       'bbl', 'health'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65218 entries, 0 to 65217
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tree_id           65218 non-null  int64  
 1   block_id          65218 non-null  int64  
 2   created_at        65218 non-null  object 
 3   tree_dbh          65218 non-null  int64  
 4   stump_diam        65218 non-null  int64  
 5   curb_loc          65218 non-null  object 
 6   status            65218 non-null  object 
 7   spc_latin         65218 non-null  object 
 8   spc_common        65218 non-null  object 
 9   steward           16421 non-null  object 
 10  guards            7957 non-null   object 
 11  sidewalk          65218 non-null  object 
 12  user_type         65218 non-null  object 
 13  problems          22542 non-null  object 
 14  root_stone        65218 non-null  object 
 15  root_grate        65218 non-null  object 
 16  root_other        65218 non-null  object

In [ ]:
df

NameError: name 'df' is not defined

In [ ]:
# Список колонок для удаления
DROP_COLUMNS = ['status', 'state', 'tree_id', 'created_at', 'stump_diam', 'address', 'spc_common', 'borocode',
                'x_sp', 'y_sp', 'council_district', 'census_tract', 'nta_name', 'bin', 'zip_city', 'community_board', 'bbl']

class InputData(BaseModel):
    tree_id: int